# MLP Training Module

In [1]:
import os
os.chdir('genre_classification_289a/src')

In [2]:
import torch
import torch.nn as nn
import os
from model import STN, MLP
import torch.optim as optim
import datetime

In [3]:
import numpy as np

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
from features import get_data_loaders, FramedFeatureDataset, FeatureDataset, DatasetSettings

### MLP Training

In [6]:
#MLP target always genre
agfs = [] #'subgenre', 'mfcc'
genre = True #False if not genre STN
    
#dataset
dataset_name = 'fma_medium'

In [7]:
settings = DatasetSettings(dataset_name, 'fma_metadata')
dataset = FramedFeatureDataset(settings,  agfs=agfs, genre=genre)
print("Num genres: ", settings.num_genres)
print(settings.genre_counts)

Num genres:  16
Rock                   6911
Electronic             6110
Experimental           2207
Hip-Hop                2109
Folk                   1477
Instrumental           1280
Pop                    1129
International          1004
Classical               598
Old-Time / Historic     510
Jazz                    380
Country                 178
Soul-RnB                154
Spoken                  118
Blues                    72
Easy Listening           21
Name: genre_top, dtype: int64


In [8]:
def get_stn_path(dataset, target):
    return '../models/DCNN_{}_{}'.format(dataset, target)

# load STNs
# @NOTE: order them by ['subgenres', 'mfcc', 'genre'] in order for analysis plots to automatically work
targets = ['subgenres', 'mfcc']
stns = [torch.load(get_stn_path(dataset_name, target)).to(device) for target in targets]
stn_layer_dims = [None, 16, 32, 64, 64, 128, 256, 256]

#which layer to extract features from
layer = 5

# setup MLP on GPU
mlp_input_size = len(targets) * stn_layer_dims[layer]
mlp_output_size = settings.num_genres
mlp = MLP(mlp_input_size, mlp_output_size)
mlp.to(device)
mlp = nn.DataParallel(mlp)

## Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
epochs = 5
batch_size = 64
valid_split = 0.2

trainloader, validloader = get_data_loaders(dataset, batch_size, valid_split)

In [9]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

def validate():
    
    with torch.no_grad():
        for stn in stns:
            stn.eval()
        mlp.eval()
                
        all_pred = []
        all_true = []
        
        for i, data in enumerate(validloader, 0):
            inputs, labels = data[0].to(device), data[1]['genre'].to(device)
            
            out_intermediate = [stn.module.forward_intermediate(inputs, layer) for stn in stns]
            input_mlp = torch.cat(out_intermediate, dim=1)
            
            out = mlp(input_mlp)
            loss = F.cross_entropy(out, labels)
            
            all_pred.append(out.argmax(dim=1))
            all_true.append(labels)
            
        all_pred = torch.cat(all_pred)
        all_true = torch.cat(all_true)
        
        curr_f1 = f1_score(all_true.cpu(), all_pred.cpu(), average='micro')
        return curr_f1

In [ ]:
#Train it
%time
losses = []
accs = []
for stn in stns:
    stn.eval()

print("Starting to train at: ", datetime.datetime.now(), " (time is +7 w.r.t. our timezone)")

#f.write('Initial Validation F1: %.6f' % validate())

mlp.train()

for epoch in range(epochs):  # loop over the dataset multiple times
    
    print('Starting epoch %d' % (epoch+1))
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1]['genre'].to(device)  #data[1]['{argument for agf being trained}']
        
        input_mlp = None
        with torch.no_grad():
            out_intermediates = [stn.module.forward_intermediate(inputs, layer) for stn in stns]
            input_mlp = torch.cat(out_intermediates, dim=1)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mlp(input_mlp)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            avg_loss = running_loss / 30
            print('[%d, %5d] loss: %.3f @ %s' % (epoch + 1, i + 1, avg_loss, datetime.datetime.now()))
            losses.append(avg_loss)
            running_loss = 0.0

print('Finished Training')

final_f1 = validate()
np.array(losses).tofile(f'logs/losses_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layer}')
np.array(final_f1).tofile(f'logs/final_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layer}')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs
Starting to train at:  2020-05-08 18:22:48.968027  (time is +7 w.r.t. our timezone)
Starting epoch 1
[1,    30] loss: 1.626 @ 2020-05-08 18:23:07.340747
[1,    60] loss: 1.257 @ 2020-05-08 18:23:23.157753
[1,    90] loss: 1.222 @ 2020-05-08 18:23:37.433896
[1,   120] loss: 1.225 @ 2020-05-08 18:23:55.625739
[1,   150] loss: 1.147 @ 2020-05-08 18:24:11.305441
[1,   180] loss: 1.156 @ 2020-05-08 18:24:27.119593
[1,   210] loss: 1.123 @ 2020-05-08 18:24:42.451626
[1,   240] loss: 1.158 @ 2020-05-08 18:25:00.212310
[1,   270] loss: 1.141 @ 2020-05-08 18:25:16.343735
[1,   300] loss: 1.147 @ 2020-05-08 18:25:34.577268
[1,   330] loss: 1.114 @ 2020-05-08 18:25:50.784629
[1,   360] loss: 1.081 @ 2020-05-08 18:26:09.336275
[1,   390] loss: 1.096 @ 2020-05-08 18:26:28.114485
[1,   420] loss: 1.112 @ 2020-05-08 18:26:44.347449
[1,   450] loss: 1.107 @ 2020-05-08 18:27:01.045681


### Save Model

In [ ]:
model_file = f'../models/MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layer}'
torch.save(mlp, model_file)

## Load & Eval Model

In [ ]:
model_file = f'../models/MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layer}'
mlp = torch.load(model_file)

Load losses and final accuracy:

In [ ]:
losses = np.fromfile(f'logs/losses_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layer}')
final_f1 = np.fromfile(f'logs/final_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layer}')

In [ ]:
final_f1

## MLP Performance History
* SGM layer 4: 0.67209446
* 